In [1]:
import pandas as pd
import random
import tensorflow as tf
import numpy as np
from tensorflow_federated import python as tff
from collections import OrderedDict
from tensorflow import keras
import os
import random
import linecache
import nest_asyncio
nest_asyncio.apply()
import time
from os import listdir
from os.path import isfile, join

2023-02-28 18:21:15.382628: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
NUM_EPOCHS = 4
# it used to be 4, reduced for debugging
BATCH_SIZE = 40
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10
time_steps =48*3
# *3
interval = 20000 # DO NOT CHANGE!!!!! it was 1000 - this is roughly 80% of 25727
future_steps = 12
split = 0.8

In [40]:
def preprocess(dataset):

    def batch_format_fn(x_d, y_d):
        return OrderedDict(
            x=x_d,
            y=y_d)

    return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

def read_subset(file,lower,upper):
    data = []
    for i in range(lower,upper):
        line = linecache.getline(file, i)
        line = line.strip('\n')
        data.append(line)
    return data  

def check_data(data):
    if 'Null' in data:
        return False
    else:
        for i in range(len(data)):
            data[i]=float(data[i])
        return data
    
def create_dataset_fed(files, lower, upper, time_steps=1):
    xs, ys = [], []
    for file in files:
        x_t, y_t = [], []
        data = read_subset(f'./ExperementData/{file}', lower, upper)
        if data:
            for i in range(len(data) - time_steps - 1 - future_steps):
                v = data[i:(i + time_steps)] 
                z = data[(i + time_steps):(i + time_steps + future_steps)]
                if check_nulls(z) and check_nulls(v):
                    x_t.append(conv_float(v))
                    y_t.append(conv_float(z))
            x_t = np.array(x_t)[:,:,np.newaxis]
            y_t = np.array(y_t)[:,:,np.newaxis]
            xs.append(x_t)
            ys.append(y_t)
    xs = np.array(xs)
    ys = np.array(ys)
    print("randomrandomrandom")
    print(xs, "\n this is ys:\n", ys)
    #return [tf.data.Dataset.from_tensor_slices((Xs[x],  np.array(ys[x]))) for x in range(len(Xs))]
#     return [ tf.data.Dataset.from_tensor_slices((Xs[x],  np.array(ys[x]))) for x in range(len(Xs))] - removed brackets, and np.array
    return [ tf.data.Dataset.from_tensor_slices((xs[x], ys[x])) for x in range(len(xs))]

def make_federated_data(files, lower, upper):
    data = create_dataset_fed(files, lower, upper,time_steps)
    return [preprocess(x) for x in data if x]

def create_keras_model():
    return tf.keras.models.Sequential([
      keras.layers.LSTM(64, input_shape=(time_steps, 1)),
      keras.layers.Dense(12),
    ])

def model_fn():
    # We _must_ create a new model here, and _not_ capture it from an external
    # scope. TFF will call this within different graph contexts.
    keras_model = create_keras_model()
    return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()])

def check_nulls(data):
    if not(all(is_float(ele) for ele in data)):
        return False
    else:
        return True
    
def conv_float(data):
    for i in range(len(data)):
        data[i]=float(data[i])
    return(data)
    
def is_float(element):
    try:
        float(element)
        return True
    except ValueError:
        return False

In [24]:
s = [1,2,3]
s = np.array(s)
print(s)
print(np.array(s))

[1 2 3]
[1 2 3]


In [36]:
# using tf.data.Dataset.from_tensor_slices() method
gfg = tf.data.Dataset.from_tensor_slices((np.array([1,2,3]), np.array([4,5])))
  
for ele in gfg:
    print(ele.numpy())

ValueError: Dimensions 3 and 2 are not compatible

In [10]:
# clusters = [['1086.txt',
#   '1466.txt',
#   '2521.txt',
#   '2706.txt',
#   '3418.txt',
#   '3766.txt',
#   '3780.txt',
#   '4342.txt',
#   '4690.txt',
#   '4927.txt',
#   '5062.txt',
#   '5445.txt',
#   '5618.txt',
#   '5720.txt',
#   '5844.txt',
#   '5932.txt',
#   '7116.txt'],
#  ['2974.txt'],
#  ['1448.txt',
#   '1632.txt',
#   '1758.txt',
#   '1985.txt',
#   '3757.txt',
#   '4968.txt',
#   '5583.txt',
#   '7136.txt'],
#  ['3676.txt', '5751.txt', '6044.txt', '7353.txt']]
# use min three (have at least one with multple properties)
clusters = [['4154.txt'], 
            ['1285.txt', 
             '3669.txt', 
             '1568.txt', 
             '3028.txt', 
             '3936.txt', 
             '3885.txt'], 
            ['2822.txt', 
             '3490.txt', 
             '1278.txt', 
             '1652.txt', 
             '2614.txt', 
             '3394.txt', 
             '2471.txt', 
             '2890.txt', 
             '2460.txt', 
             '3421.txt', 
             '1039.txt', 
             '4009.txt', 
             '3001.txt', 
             '3375.txt', 
             '3406.txt', 
             '2907.txt', 
             '1360.txt', 
             '2119.txt', 
             '3672.txt', 
             '3869.txt', 
             '2780.txt', 
             '3847.txt', 
             '3338.txt'], 
            ['3650.txt', 
             '4063.txt', 
             '3486.txt', 
             '1086.txt', 
             '1733.txt', 
             '1901.txt', 
             '2211.txt', 
             '3182.txt', 
             '1838.txt', 
             '2885.txt', 
             '3769.txt', 
             '2492.txt', 
             '1170.txt', 
             '2096.txt', 
             '2733.txt', 
             '2041.txt', 
             '3822.txt', 
             '2137.txt', 
             '2494.txt', 
             '1149.txt', 
             '2118.txt', 
             '1516.txt', 
             '3843.txt', 
             '3060.txt', 
             '1716.txt', 
             '2555.txt', 
             '1706.txt', 
             '3850.txt', 
             '2623.txt', 
             '1513.txt']]



In [41]:
randomDict = make_federated_data(clusters[1], 1, 1+interval)


/var/folders/by/4rt6rrtn24n6r36ktnkzt6q00000gn/T/ipykernel_86262/222881433.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  xs = np.array(xs)
/var/folders/by/4rt6rrtn24n6r36ktnkzt6q00000gn/T/ipykernel_86262/222881433.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ys = np.array(ys)


ESIESIESI
[array([[[0.985],
         [1.006],
         [0.949],
         ...,
         [1.047],
         [1.066],
         [1.042]],

        [[1.006],
         [0.949],
         [0.955],
         ...,
         [1.066],
         [1.042],
         [1.066]],

        [[0.949],
         [0.955],
         [0.996],
         ...,
         [1.042],
         [1.066],
         [1.07 ]],

        ...,

        [[1.271],
         [2.972],
         [2.64 ],
         ...,
         [0.822],
         [0.766],
         [0.698]],

        [[2.972],
         [2.64 ],
         [2.383],
         ...,
         [0.766],
         [0.698],
         [2.238]],

        [[2.64 ],
         [2.383],
         [2.618],
         ...,
         [0.698],
         [2.238],
         [2.213]]]) array([[[1.499],
                            [1.405],
                            [1.411],
                            ...,
                            [2.56 ],
                            [1.264],
                            [1.126

In [11]:
example_dataset = create_dataset_fed(['1086.txt'],1, 500,time_steps)[0]
preprocessed_example_dataset = preprocess(example_dataset)
# process = [tff.learning.build_federated_averaging_process - old code, had to update it
process = [tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: keras.optimizers.Adam(0.001),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)) for _ in range(len(clusters))]
          
state = [process[x].initialize() for x in range (len(process))]
for c in range(len(clusters)):
    max_time = (len(clusters[c])/60)*9000
#     CHANGE THIS TO 9000 and 30
    start_time = time.time()
    while time.time() -start_time < max_time:
        selected_files = clusters[c]
        print(selected_files)
        location = random.randint(1,int(25727*split)-interval)
        print(location)
        federated_train_data = make_federated_data(selected_files, location, location+interval)
        if len(federated_train_data)>0:
#             state[c], metrics = process[c].next(state[c], federated_train_data) - old code, does not iterate
#             is the below the right thing for state? It does not work for state[c] as this is not iterable
            result = process[c].next(state[c], federated_train_data)
            state[c] = result.state
            metrics = result.metrics
            print('round {:2d}, metrics={}'.format(c, metrics))
#     tensorflow 2.9.0 and federated 0.40.0
#             metrics = process[c].next(state[c], federated_train_data).metrics
#             print('metrics={}'.format(metrics['client_work']['train']))
#             print('metrics={}'.format( metrics['train'])) - old code, not the right format

['4154.txt']
112
round  0, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('root_mean_squared_error', 3.7621424), ('loss', 14.153701), ('num_examples', 27544), ('num_batches', 689)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
['4154.txt']
42
round  0, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('root_mean_squared_error', 2.4776938), ('loss', 6.1389623), ('num_examples', 27824), ('num_batches', 696)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
['4154.txt']
349
round  0, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('root_mean_squared_error', 2.298397), ('loss', 5.2826304), ('num_examples', 26596), ('num_batches', 665)]))])), ('aggregator', OrderedDict([('mean_valu

/var/folders/by/4rt6rrtn24n6r36ktnkzt6q00000gn/T/ipykernel_86262/3661235097.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Xs = np.array(Xs)
/var/folders/by/4rt6rrtn24n6r36ktnkzt6q00000gn/T/ipykernel_86262/3661235097.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ys = np.array(ys)


round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('root_mean_squared_error', 1.2926028), ('loss', 1.6708226), ('num_examples', 398508), ('num_batches', 9965)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
['1285.txt', '3669.txt', '1568.txt', '3028.txt', '3936.txt', '3885.txt']
476
round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('root_mean_squared_error', 1.216207), ('loss', 1.4791601), ('num_examples', 398012), ('num_batches', 9953)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
['1285.txt', '3669.txt', '1568.txt', '3028.txt', '3936.txt', '3885.txt']
69
round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('root_mean_squared_error', 1.2299725), ('loss

In [12]:
data = []
onlyfiles = [f for f in listdir('./ExperementData/') if isfile(join('./ExperementData/', f))and f[-4:]==".txt"]
for f in onlyfiles:
    with open ('./ExperementData/'+f,'r') as reader:
        temp = []
        for line in reader:
            if line.strip() == 'Null':
                temp.append('Null')
            else:
                temp.append(float(line.strip()))
        data.append(temp)
test = [d[int(25727*split):] for d in data]
# last 20% of every file
alloc = {}
for c in range(len(clusters)):
    for i in clusters[c]:
        alloc[i]=c


In [13]:
alloc

{'4154.txt': 0,
 '1285.txt': 1,
 '3669.txt': 1,
 '1568.txt': 1,
 '3028.txt': 1,
 '3936.txt': 1,
 '3885.txt': 1,
 '2822.txt': 2,
 '3490.txt': 2,
 '1278.txt': 2,
 '1652.txt': 2,
 '2614.txt': 2,
 '3394.txt': 2,
 '2471.txt': 2,
 '2890.txt': 2,
 '2460.txt': 2,
 '3421.txt': 2,
 '1039.txt': 2,
 '4009.txt': 2,
 '3001.txt': 2,
 '3375.txt': 2,
 '3406.txt': 2,
 '2907.txt': 2,
 '1360.txt': 2,
 '2119.txt': 2,
 '3672.txt': 2,
 '3869.txt': 2,
 '2780.txt': 2,
 '3847.txt': 2,
 '3338.txt': 2,
 '3650.txt': 3,
 '4063.txt': 3,
 '3486.txt': 3,
 '1086.txt': 3,
 '1733.txt': 3,
 '1901.txt': 3,
 '2211.txt': 3,
 '3182.txt': 3,
 '1838.txt': 3,
 '2885.txt': 3,
 '3769.txt': 3,
 '2492.txt': 3,
 '1170.txt': 3,
 '2096.txt': 3,
 '2733.txt': 3,
 '2041.txt': 3,
 '3822.txt': 3,
 '2137.txt': 3,
 '2494.txt': 3,
 '1149.txt': 3,
 '2118.txt': 3,
 '1516.txt': 3,
 '3843.txt': 3,
 '3060.txt': 3,
 '1716.txt': 3,
 '2555.txt': 3,
 '1706.txt': 3,
 '3850.txt': 3,
 '2623.txt': 3,
 '1513.txt': 3}

In [14]:
def create_dataset(data, time_steps=1):
    Xs, ys = [], []
    for i in range(len(data) - time_steps-12):
        v = data[i:(i + time_steps)]
        z = data[(i + time_steps):(i + time_steps+12)]
        if check_nulls(z) and check_nulls(v):
            ys.append(z)
            Xs.append(v)
    return np.array(Xs), np.array(ys)

In [22]:
# NEED TO UNDERSTAND WHY THERE ARE NULLS
# WHY DO I HAVE A RESULT FOR 60 FILES AND NOT 12 - we train on all files but 80% of the readings and we test on all the houses in experment data but we test on the rest of the 20% of the readings
# why is the training so weird? - trains per cluster and updates the weights in the alloted time so three times per cluster 

print(len(test))
for i in range(len(test)):
    X_test, y_test = create_dataset(test[i], time_steps)
    print("HEREEeeeeeeeeeee")
    print(len(X_test),len(y_test) )
    print(X_test, "ihbserflijbenlihfb\n", y_test)
    if X_test.size > 0 and y_test.size > 0:
        X_test = X_test[:,:,np.newaxis]
        y_test = y_test[:,:,np.newaxis]
        model_for_inference = create_keras_model()
        weights = state[alloc[onlyfiles[i]]].global_model_weights
    #     weights = process[c].get_model_weights(state[alloc[onlyfiles[i]]]) = guesses
        weights.assign_weights_to(model_for_inference)
    #     state[alloc[onlyfiles[i]]].model.assign_weights_to(model_for_inference) = old code

    #     process.get_model_weights(state[alloc[onlyfiles[i]]]) = what i found online

    #     assigning weights from fed model to new keras model for prediciton
        y_pred = model_for_inference.predict(X_test)
        dataframe = pd.DataFrame(np.squeeze(np.array(y_pred)))
    #     dataframe.to_csv(r"./fed_weight_clust/pred/"+onlyfiles[i][:4]+'.csv')
        dataframe.to_csv(r"./num_clust_four/pred-fuzzycmeans/"+onlyfiles[i][:4]+'.csv')
        dataframe = pd.DataFrame(np.squeeze(np.array(y_test)))
    #     dataframe.to_csv(r"./fed_weight_clust/test/"+onlyfiles[i][:4]+'.csv')
        dataframe.to_csv(r"./num_clust_four/test-fuzzycmeans/"+onlyfiles[i][:4]+'.csv')
    
    
    
#     created new keras model then assiging weights from other model to this one
#     find new method for fed learning and making new predicitions (.predict or smth)
# .predict
# state is state of fed model that has been trained (model for prediction)

60
HEREEeeeeeeeeeee
4990 4990
[[0.142 0.261 0.297 ... 0.124 0.137 0.135]
 [0.261 0.297 0.473 ... 0.137 0.135 0.12 ]
 [0.297 0.473 0.497 ... 0.135 0.12  0.191]
 ...
 [0.438 0.288 0.278 ... 0.205 0.965 0.71 ]
 [0.288 0.278 0.609 ... 0.965 0.71  1.247]
 [0.278 0.609 0.771 ... 0.71  1.247 0.796]] ihbserflijbenlihfb
 [[0.12  0.191 0.155 ... 0.425 0.428 0.34 ]
 [0.191 0.155 0.549 ... 0.428 0.34  0.18 ]
 [0.155 0.549 0.423 ... 0.34  0.18  0.156]
 ...
 [1.247 0.796 0.694 ... 0.567 0.568 0.169]
 [0.796 0.694 0.605 ... 0.568 0.169 0.167]
 [0.694 0.605 0.455 ... 0.169 0.167 0.193]]
156/156 [==============================] - 5s 28ms/step
HEREEeeeeeeeeeee
4787 4787
[[0.292 0.231 0.478 ... 0.252 0.252 0.255]
 [0.231 0.478 0.673 ... 0.252 0.255 0.315]
 [0.478 0.673 0.308 ... 0.255 0.315 0.335]
 ...
 [0.165 0.292 0.531 ... 0.287 0.524 0.367]
 [0.292 0.531 0.512 ... 0.524 0.367 0.365]
 [0.531 0.512 0.548 ... 0.367 0.365 0.303]] ihbserflijbenlihfb
 [[0.315 0.335 0.326 ... 0.256 0.257 0.258]
 [0.335 0.32

KeyboardInterrupt: 

In [18]:
print("hello")

hello
